# ArcFace: Additive Angular Margin Loss for Deep Face Recognition.  
[click here](https://arxiv.org/abs/1801.07698)  

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import math
from tensorflow.keras import backend as K
import os
from skimage.transform import resize
from skimage.io import imread

In [2]:
class My_Custom_Generator(tf.keras.utils.Sequence) :
  
  def __init__(self, X, labels, batch_size) :
    self.X = X
    self.labels = labels
    self.batch_size = batch_size
    self.shuffle = True
    self.on_epoch_end()

  def __iter__(self):
    """Create a generator that iterate over the Sequence."""
    for item in (self[i] for i in range(len(self))):
      yield item  
    
  def __len__(self) :
    #self.on_epoch_end()
    result = (np.ceil(len(self.X) / float(self.batch_size))).astype(np.int)
    return result

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.labels))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)
  
  def __getitem__(self, idx) :
    indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
    batch_x = [self.X[k] for k in indexes]
    batch_y = [self.labels[k] for k in indexes] 
    X = np.array(batch_x, dtype=np.int32)
    Y = np.array(batch_y)
    X = np.concatenate((X, Y), axis=1)
     
    return X, Y



class My_Custom_GeneratorURLs(tf.keras.utils.Sequence) :
  
  def __init__(self, image_filenames, labels, batch_size, height, width, num_channels = 3) :
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size
    self.shuffle = True
    self.on_epoch_end()
    self.height = height
    self.width = width
    self.num_channels = num_channels

  def __iter__(self):
    """Create a generator that iterate over the Sequence."""
    for item in (self[i] for i in range(len(self))):
      yield item  
    
  def __len__(self) :
    #self.on_epoch_end()
    result = (np.floor(len(self.image_filenames) / float(self.batch_size))).astype(np.int)  # do not  use ceil  because the histogram don know
    return result

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.labels))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)
  
  def __getitem__(self, idx) :
    #global height_reshaped, width_reshaped
    indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
    b_size = len(indexes)

    batch_x = [self.image_filenames[k] for k in indexes] 
    batch_y = [self.labels[k] for k in indexes]
    #  255.0 * resize(imread(str(file_name)), (height_reshaped, width_reshaped, 3))
    X = np.array([ 255.0 * imread(str(file_name))     #
               for file_name in batch_x], dtype=np.int32)
    
    Y = np.array(batch_y)
    X = np.reshape(X, (b_size, self.num_channels * self.height * self.width))
    Y = np.reshape(Y, (-1,1))
    X = np.concatenate((X, Y), axis=1)
    return X, Y 


In [3]:
class ArcFace1D(layers.Layer):
    def __init__(self, n_classes=10, s=30.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace1D, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = tf.keras.regularizers.get(regularizer)

    def build(self, input_shape):
        super(ArcFace1D, self).build(input_shape)
        #print('wwwwwwwww')
        #print(input_shape)
        #print(input_shape[-1])
        self.W = self.add_weight(name='W',
                                shape=(input_shape[-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, x, y):
        #x, y = inputs
        c = K.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        # clip logits to prevent zero division when backward
        theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
        target_logits = tf.cos(theta + self.m)
        # sin = tf.sqrt(1 - logits**2)
        # cos_m = tf.cos(logits)
        # sin_m = tf.sin(logits)
        # target_logits = logits * cos_m - sin * sin_m
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)

class ArcFace(layers.Layer):
    def __init__(self, n_classes=10, s=30.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = tf.keras.regularizers.get(regularizer)

    def build(self, input_shape):
        super(ArcFace, self).build(input_shape)
        self.W = self.add_weight(name='W',
                                shape=(input_shape[-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, x, y):
        #x, y = inputs
        c = K.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        # clip logits to prevent zero division when backward
        theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
        target_logits = tf.cos(theta + self.m)
        # sin = tf.sqrt(1 - logits**2)
        # cos_m = tf.cos(logits)
        # sin_m = tf.sin(logits)
        # target_logits = logits * cos_m - sin * sin_m
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)


In [4]:
num_patterns = 2000
num_features = 2
mu, sigma = 0, 0.5
num_classes = 2
num_hidden_neurons = 4
Xred = np.random.normal(mu, sigma, (num_patterns, num_features)) + np.array([1,1])
Yred = np.zeros(num_patterns, dtype=int)
Xblue = np.random.normal(mu, sigma, (num_patterns, num_features)) + np.array([-1,-1])
Yblue = np.ones(num_patterns, dtype=int)
X = np.concatenate((Xred, Xblue), axis=0)
Y = np.concatenate((Yred, Yblue), axis=0)
Y = np.reshape(Y, (-1,1))
print(X.shape)
print(Y.shape)
print(Y)

(4000, 2)
(4000, 1)
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]


In [5]:
inputs = tf.keras.Input(shape=(num_features+num_classes))
output_1 = layers.Dense(4, activation='relu')(inputs[:,0:num_features])
output_2 = layers.Dense(4, activation='relu')(output_1)
predictions = ArcFace1D(num_classes)(output_2, inputs[:,num_features: num_features+num_classes])  
model = tf.keras.Model(inputs, predictions)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=34)
batch_size = 100
num_epochs = 50
my_training_batch_generator = My_Custom_Generator(X_train, y_train, batch_size)
my_validation_batch_generator = My_Custom_Generator(X_test, y_test, batch_size)

model.compile(
  'adam',
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=['accuracy']
)

model.fit(
  my_training_batch_generator,
  epochs=num_epochs,
  validation_data = my_validation_batch_generator,
  validation_steps=len(my_validation_batch_generator)  
)

Epoch 1/50
36/36 [==============================] - 1s 24ms/step - loss: 2.2766 - accuracy: 0.6558 - val_loss: 1.3260 - val_accuracy: 0.6225
Epoch 2/50
36/36 [==============================] - 0s 6ms/step - loss: 0.9351 - accuracy: 0.4839 - val_loss: 0.7449 - val_accuracy: 0.3750
Epoch 3/50
36/36 [==============================] - 0s 5ms/step - loss: 0.7278 - accuracy: 0.3331 - val_loss: 0.7242 - val_accuracy: 0.4100
Epoch 4/50
36/36 [==============================] - 0s 6ms/step - loss: 0.7196 - accuracy: 0.3697 - val_loss: 0.7189 - val_accuracy: 0.3575
Epoch 5/50
36/36 [==============================] - 0s 5ms/step - loss: 0.7162 - accuracy: 0.3786 - val_loss: 0.7163 - val_accuracy: 0.2725
Epoch 6/50
36/36 [==============================] - 0s 4ms/step - loss: 0.7141 - accuracy: 0.3450 - val_loss: 0.7145 - val_accuracy: 0.2675
Epoch 7/50
36/36 [==============================] - 0s 4ms/step - loss: 0.7126 - accuracy: 0.3136 - val_loss: 0.7132 - val_accuracy: 0.2675
Epoch 8/50
36/36 [=

In [7]:
classes_texts = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes_texts)
height = 32
width = 32
num_channels = 3
batch_size = 8
X_train = list(((os.path.abspath('./../../assets/cifar10small/'+ name+str(i+1)+'.png')) for name in classes_texts for i in range(7)))
X_test  = list(((os.path.abspath('./../../assets/cifar10small/'+ name+str(7+i+1)+'.png')) for name in classes_texts for i in range(3))) 
y_train = list((j for j in range(10) for i in range(7)))
y_test = list((j for j in range(10) for i in range(3)))

my_training_batch_generator_cifar = My_Custom_GeneratorURLs(X_train, y_train, batch_size, height, width)
my_validation_batch_generator_cifar = My_Custom_GeneratorURLs(X_test, y_test, batch_size, height, width)


In [8]:

num_features = height* width* num_channels
inputs = tf.keras.Input(shape=(num_features+num_classes))
output_1 = layers.Dense(44, activation='relu')(inputs[:,0:num_features])
output_2 = layers.Dense(44, activation='relu')(output_1)
predictions = ArcFace1D(num_classes)(output_2, inputs[:,num_features: num_features+num_classes])  
model = tf.keras.Model(inputs, predictions)

In [9]:
num_epochs = 50
model.compile(
  'adam',
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy']
)

model.fit(
  my_training_batch_generator_cifar,
  epochs=num_epochs,
  validation_data = my_validation_batch_generator_cifar,
  validation_steps=len(my_validation_batch_generator)  
)

Epoch 1/50
8/8 [==============================] - 1s 117ms/step - loss: 327.0363 - accuracy: 0.0000e+00 - val_loss: 396.8424 - val_accuracy: 0.0000e+00
Epoch 2/50
8/8 [==============================] - 0s 36ms/step - loss: 405.0330 - accuracy: 0.0000e+00
Epoch 3/50
8/8 [==============================] - 0s 23ms/step - loss: 394.1513 - accuracy: 0.0156
Epoch 4/50
8/8 [==============================] - 0s 22ms/step - loss: 450.7028 - accuracy: 0.0000e+00
Epoch 5/50
8/8 [==============================] - 0s 20ms/step - loss: 459.8130 - accuracy: 0.0000e+00
Epoch 6/50
8/8 [==============================] - 0s 22ms/step - loss: 481.4299 - accuracy: 0.0000e+00
Epoch 7/50
8/8 [==============================] - 0s 17ms/step - loss: 494.4612 - accuracy: 0.0469
Epoch 8/50
8/8 [==============================] - 0s 14ms/step - loss: 492.8427 - accuracy: 0.0938
Epoch 9/50
8/8 [==============================] - 0s 16ms/step - loss: 531.9174 - accuracy: 0.1094
Epoch 10/50
8/8 [=======================

In [10]:
X1 = np.random.random((100,3,64,64))
X1 = np.reshape(X1, (100,3*64*64))
Y = np.random.random((100,10))
X1 = np.concatenate((X1, Y), axis=1)

X2 = X1[:,0: 3*64*64]
Y2 = X1[:,3*64*64:3*64*64+10]
print(X1.shape)
print(X2.shape)
print(Y2.shape)

X2 = np.reshape(X2,[-1,3,64,64])
print(X2.shape)

(100, 12298)
(100, 12288)
(100, 10)
(100, 3, 64, 64)
